In [1]:
# Generelle moduler og funksjonsbeskrivelser brukt i forelesningen
from numpy import sin, cos, pi, exp
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
from Kildekode._12_IIRFilt import *
from Kildekode._14_Z_transformasjon import *

%matplotlib ipympl

<img src="NTNU_Logo.png" align="left" style="width: 30%">
<br clear="all" />
<br></br>

# Z-transformasjon

* **Emne IELEA2302 - Signalbehandling**
* **Uke 14, 2021**
* **Underviser: Kai Erik Hoff**

## Tema Tirsdag 6. april:
* Repetisjon IIR-filtre
* Fra differanseligning til frekvensrespons
* z-transformasjon
    * Fourier og laplace (for kontekst)
    * Z-transformasjon
    * Z-planet
* Transferfunksjon $H(z)$

# Tema Fredag 9. april:
* Repetisjon z-transformasjon
* Repetisjon Transferfunksjon
* Poler og nullpunkt
* Pol- og nullpunktskart
* Poler, nullpunkt og frekvensrespons (filterform)

# Tema Tirsdag 13. april:

* Repetisjon transferfunksjon, poler og nullpunkt
* Fortsettele poler, nullpunkt og frekvensrespons (filterform)
* Stabilitet
* Quiz

## Repetisjon IIR filtre

* **IIR** - **I**nfinite **I**mpulse **R**esponse
* Fellesbetegnelse for filter som kalkulerer utgangsveriden $y[n]$ basert på en lineær kombinasjon av verdiene til inngangssignalet ved foregående sampletidspunkt $x[n-k], \ \ k\in \{0, 1, 2, \ldots\}$ ***og*** tidligere kalkulerte verdier til utgangssignalet $y[n-k], \ \ k \in \{1, 2, \ldots\}$.


* **Differanseligning:**
$$\sum_{k=0}^{N} a_k \cdot y[n-k] = \sum_{k=0}^{M} b_k \cdot x[n-k]$$
* **Filteralgoritme:**
$$y[n] = \frac{1}{a_0}\cdot \left(\sum_{k=0}^{M} b_k \cdot x[n-k] - \sum_{k=1}^{N}a_k\cdot  y[n-k] \right)$$

## Repetisjon Rekursiv filterstruktur

* IIR filtre omtales ofte som *rekursive* filtre ettersom utgangssignalet tilbakekobles som input.
<img src="Figurer/12_IIR%20Filtre/Fig0_Recursion.png" style="width: 35%; margin-left: 400px" />

* Impulsresponsen $h[n]$ for et slikt filter vil ha uendelig varighet.
* Impulsrespons og filterkoeffisienter er ikke lengre synonyme.
$$b_k \neq h[n]\big{|}_{n=k}$$

## Repetisjon IIR filtre

* Ved å lage et *rekursivt* filter, kan selv enkle førsteordens filtre produsere en langvarig impulsrespons.
* Et IIR-filter med relativt lav filterorden kan, dersom vi velger filterkoeffisienter med omhu, oppfylle de samme kravene for amplitduerespons som et FIR filter med ***mye*** høyere filterorden.
* Ulempen er at IIR filtre ***ikke*** har lineær faserespons i passbåndet. 
    * Konsekvensen av dette er *tidsforskyvningen* for hver av frekvenskomponentene til det filtrerte signalet gitt i antall sampler ikke vil være konstant.
    * Dette har potensiale til å forårsake uønsket distorsjon av f.eks. et lydsignal.

## Impulsrespons førsteordens IIR-filter

* Eksempel:
$$y[n] = x[n] + 0.9\cdot y[n-1]$$
* Ettersom et IIR-filter tar i bruk tidligere utgangsverdier får vi en *geometrisk rekke*:
|$$n$$|$$x[n]=\delta[n]$$|$$y[n]=h[n]$$|
|:-:|:-:|:-:|
|0|1|$$y[0] = 1 + 0.9\cdot 0 = 1$$|
|1|0|$$y[1] = 0 + 0.9\cdot 1 = 0.9$$|
|2|0|$$y[2] = 0 + 0.9\cdot 0.9 = 0.81$$|
|3|0|$$y[3] = 0 + 0.9\cdot 0.81 = 0.9^3$$|
|4|0|$$y[3] = 0 + 0.9\cdot 0.9^3 = 0.9^4$$|
|$$\ldots$$|$$\ldots$$|$$\ldots$$|

$$h[n] = u[n] \cdot 0.9^n$$

## Repetisjon Regneeksempel 3: 
* Gitt høypassfilteret
$$y[n] = x[n]-x[n-1]-0.8\cdot y[n-1]$$
Finn et uttrykk for frekvensresponsen, og korriger deretter filterkoeffisientene slik at $\text{max}\left(\left|H\left(e^{j\hat{\omega}}\right)\right|\right) = 1$.

In [2]:
# Frekvensresponsen til filteret i eksempel 3
FreqRespDemo([0.1, -0.1], [1, 0.8], fig_num=1);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

# Tidsforskyvning i frekvensplanet

* En sentral egenskap for utregning av frekvensrespons til digitale filtre er tidsforskyving
* Vi vet at:
<br>


\begin{align}
DTFT(\alpha\cdot x[n-D]) &= \alpha \cdot  X\left(e^{j\hat{\omega}}\right) \cdot e^{-j\hat{\omega}\cdot D}\\
&\text{ettersom}\\
x[n-D] &= x[n]*\delta[n-D]\\
&\text{og}\\
DTFT(\delta[n-D]) &=e^{-j\hat{\omega}\cdot D}
\end{align}

* Dermed kan vi fouriertransformere begge sidene av differanseligningen til et filter.

# Frekvensrespons via differanseligning



\begin{align}
DTFT\left(\sum_{k=0}^{N} a_k \cdot y[n-k]\right) &= DTFT\left(\sum_{k=0}^{M} b_k \cdot x[n-k] \right)\\
\sum_{k=0}^{N} a_k \cdot Y\left(e^{j\hat{\omega}}\right)\cdot e^{-j\hat{\omega}\cdot k} &=\sum_{k=0}^{M} b_k \cdot X\left(e^{j\hat{\omega}}\right)\cdot e^{-j\hat{\omega}\cdot k}\\
Y\left(e^{j\hat{\omega}}\right)\cdot \left(\sum_{k=0}^{N} a_k \cdot e^{-j\hat{\omega}\cdot k}\right) &= X\left(e^{j\hat{\omega}}\right)\cdot \left(\sum_{k=0}^{M} b_k \cdot e^{-j\hat{\omega}\cdot k}\right)\\
H\left(e^{j\hat{\omega}}\right) = \frac{Y\left(e^{j\hat{\omega}}\right)}{X\left(e^{j\hat{\omega}}\right)} &= \frac{\sum_{k=0}^{M} b_k \cdot e^{-j\hat{\omega}\cdot k}}{\sum_{k=0}^{N} a_k \cdot e^{-j\hat{\omega}\cdot k}}
\end{align}

## Eksempel: 

* Gitt høypassfilteret
$$y[n] = 0.1\cdot x[n]-0.1\cdot x[n-1]-0.8\cdot y[n-1]$$
kan vi finne frekvensresponsen på følgende vis:



\begin{align}
y[n] &= 0.1\cdot x[n]-0.1\cdot x[n-1]-0.8\cdot y[n-1]\\
y[n] + 0.8\cdot y[n-1] &= 0.1\cdot x[n] - 0.1\cdot x[n-1] \\
\text{DTFT}(y[n] + 0.8\cdot y[n-1]) &=\text{DTFT}(0.1\cdot x[n] - 0.1\cdot x[n-1])\\
Y\left(e^{j\hat{\omega}}\right) + 0.8\cdot e^{-j\hat{\omega}\cdot 1} \cdot Y\left(e^{j\hat{\omega}}\right) &= 0.1\cdot X\left(e^{j\hat{\omega}}\right) - 0.1\cdot e^{-j\hat{\omega}\cdot 1} \cdot X\left(e^{j\hat{\omega}}\right) \\
Y\left(e^{j\hat{\omega}}\right)\cdot \left(1+0.8\cdot e^{-j\hat{\omega}} \right) &= X\left(e^{j\hat{\omega}}\right)\cdot \left(0.1-0.1\cdot e^{-j\hat{\omega}} \right)\\
\frac{Y\left(e^{j\hat{\omega}}\right)}{X\left(e^{j\hat{\omega}}\right)} &= \frac{0.1 - 0.1\cdot e^{-j\hat{\omega}}}{1+0.8\cdot e^{-j\hat{\omega}}} = H\left(e^{j\hat{\omega}}\right)
\end{align}

# Z-transformasjon

* Kartlegger filteregenskaper i "z-planet", som har 2 dimensjoner
* Viktig for analyse og design av digitale filtre
* Kan sammenlignes med Laplacetransformasjon for tidskontinuerlige LTI-system


## Oppsummering Fouriertransformasjon *(tidskontinuerlig)*

#### Formel

$$F(j\omega) = \int_0^{\infty}f(t)\cdot e^{-j\omega t} dt$$

#### For LTI-system

* Dersom $f(t)$ er impulsresponsen til et LTI-system, forteller dette hvordan systemet vil påvirke et inngangssignal $x(t) = e^{j\omega \cdot t}$ med vinkelfrekvens $\omega$.
    * Gjelder også sinusformede signal da  $\cos(\omega\cdot t) = \frac{1}{2} e^{j\omega\cdot t} + \frac{1}{2} e^{-j \omega \cdot t}$.

# Laplacetransformasjon *(tidskontinuerlig)*

#### Formel:

\begin{align}
F(s) &= \int_0^{\infty}f(t)\cdot e^{-s\cdot t} dt\\
&\text{der}\\
s &= \sigma + j\omega
\end{align}

#### For LTI-system
* Dersom $f(t)$ er impulsresponsen til et LTI-system, forteller dette hvordan systemet vil påvirke et inngangssignal $x(t) = e^{-s \cdot t} = e^{-\sigma \cdot t} \cdot e^{-j \omega\cdot t}$.
* Refleksjoner:
    * Dersom $\sigma = 0$, reduseres den 2-dimensjonale laplacetrasformasjonen til fouriertransformasjon.
    * Dersom $\sigma > 0$, vil amplitudent til $x(t)$ synke over tid.
    * Dersom $\sigma < 0$, vil amplituden til $x(t)$ øke eksponentielt, og vi får "unbounded input".
    
<br>

In [3]:
# Angi punkt på s-planet
s = -4 + 2j*pi*8
# Regn ut e^(-st)
t = np.linspace(0, 1, 501)
x_t = exp(-s*t)

plt.close(2); plt.figure(2)
plt.subplot(2,1,1)
plt.plot(t, np.real(x_t))
plt.ylabel(r'Re($x(t)$)')
plt.grid(True)
plt.subplot(2,1,2)
plt.plot(t, np.imag(x_t))
plt.ylabel(r'Im($x(t)$)')
plt.xlabel(r't (sekund)')
plt.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Hvorfor ikke bruke laplace for digitale filtre?

* Frekvenser $|\hat{\omega}|>\pi$ er vil ikke være gyldige.
* Aliasing må tas i betraktning.
    - Løses med å transformere til "polarkoordinater".
    - Digital frekvens $\hat{\omega}$ gitt ved vinkel
    - Demping gitt ved avstand til origo.

## Oppsummering DTFT
#### Formel

$$H\left(e^{j\hat{\omega}}\right) = \sum_{n=0}^{\infty}h[n]\cdot e^{-j\hat{\omega}\cdot n}$$

#### For LTI-system

* Dersom $h[n]$ er impulsresponsen til et LTI-system, forteller dette hvordan systemet vil påvirke et inngangssignal $x[n] = e^{-j\omega \cdot n}$ med vinkelfrekvens $\hat{\omega}$.
    * Gjelder også sinusformede signal da  $\cos(\hat{\omega}\cdot n) = \frac{1}{2} e^{j\hat{\omega}\cdot n} + \frac{1}{2} e^{-j \hat{\omega} \cdot n}$.

# Z-transformasjon
#### Notasjon:
$$\textbf{Z}(x[n]) = X(z)$$

#### Formel:
\begin{align}
H(z) &= \sum_{n=0}^{\infty} h[n]\cdot z^{-n}\\
& \text{der}\\
z &= r\cdot e^{j\hat{\omega}}
\end{align}

#### For LTI-system
* Dersom $h[n]$ er impulsresponsen til et LTI-system, forteller dette hvordan systemet vil påvirke et inngangssignal $x[n] = z^{-n} = r^{-n} \cdot e^{-j \hat{\omega}\cdot n}$.
* Refleksjoner:
    * Dersom $|r| = 1$, reduseres den 2-dimensjonale Z-transformen til DTFT. 
        * Dette er grunnen til at notasjonen for DTFT er $H\left(e^{j\hat{\omega}}\right)$.
    * Dersom $|r| > 1$, vil amplitudent til $x[n]$ synke over tid.
    * Dersom $|r| < 1$, vil amplituden til $x[n]$ øke eksponentielt, og vi får "unbounded input".
    
<br>

# S-plan og Z-plan
<br>
<img src="Figurer/14_Ztransform/Fig2_SandZ.png" style="width: 80%; margin-left: 100px" />

In [4]:
# Angi punkt på z-planet i polarkoordinater
#z = 0.9*exp(2j*pi/8) # Innenfor enhetssirkel
#z = 1*exp(2j*pi/8)     # På enhetssirkel
z = 1.1*exp(2j*pi/8) # Utenfor enhetssirkel

# Regn ut z^(-n)
n = np.arange(0, 32)
x_n = z**-n

plt.close(3); plt.figure(3)
plt.subplot(2,1,1)
plt.stem(n, np.real(x_n), markerfmt='.', basefmt='grey', use_line_collection=True)
plt.ylabel(r'Re($x[n]$)')
plt.grid(True)
plt.subplot(2,1,2)
plt.stem(n, np.imag(x_n), markerfmt='.', basefmt='grey', use_line_collection=True)
plt.ylabel(r'Im($x[n]$)')
plt.xlabel(r'n (samplenummer)')
plt.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# LTI-filter i Z-domene

<br>

<img src="Figurer/14_Ztransform/Fig3_TF.png" style="width: 80%; margin-left: 100px" />

# Transferfunksjon / overføringsfunksjon

* Z-transformen av impulsresponen $h[n]$ til et LTI-system kalles systemets *transferfunksjon*
* For ethvert kausalt LTI-system med filterkoeffisienter **a** og **b**, vil transferfunksjonen være:
\begin{align}
H(z) = \frac{Y(z)}{X(z)} &= \frac{\sum_{k=0}^{M} b_k \cdot z^{-k}}{\sum_{k=0}^{N} a_k \cdot z^{-k}}\\
&\text{eller}\\
&= \frac{b_0 + b_1\cdot z^{-1} + b_2 \cdot z^{-2} + \ldots + b_M \cdot z^{-M}}{a_0 + a_1\cdot z^{-1} + a_2 \cdot z^{-2} + \ldots + a_N \cdot z^{-N}}
\end{align}
* Teller og nevner er polynomer
* *samme utledning som for DTFT av differanseligning*

<img src="Figurer/14_Ztransform/PZ_lecture/Slide1.PNG" style="width: 85%; margin-left: 50px" />

# Frekvensrespons

* Hvordan et filter påvirker et inngangssignal som er en ren sinussekvens.


<img src="Figurer/14_Ztransform/Fig7_DTFT.png" style="width: 60%; margin-left: 200px" />

# Transferfunksjon

* Hvordan et filter påvirker et inngangssignal på formen $z^{-n}$.


<img src="Figurer/14_Ztransform/Fig8_TF.png" style="width: 60%; margin-left: 200px" />

# Tidsforskyvning i Z-planet

* Hver sample tidsforskyvning tilsvarer multiplikasjon med $z^{-1}$ i z-planet. 
$$\textbf{Z}(x[n-D]) = z^{-D}\cdot X(z)$$

* I blokkskjemarepresentasjon av filter brukes derfor $z^{-1}$ for å symbolisere tidsforskyvning/delay.

<br>

<img src="Figurer/14_Ztransform/Fig4_Delay.png" style="width: 70%; margin-left: 100px" />

## Regneeksempel 1

* Finn transferfunksjonen $H(z)$ til filteret i vist i figuren:
<br>

<img src="Figurer/14_Ztransform/Fig5_ExFilt.png" style="width: 70%; margin-left: 100px" />

# Visualisering av $H(z)$

* $H(z)$ funksjon av et komplekst tall.
    * "Gain" i transferfunksjonen av størst interesse
    * Et "Surface Plot" av $|H(z)|$ vil kunne gi informasjon om forsterkningen av ulike typer signal.
    * Amplituderesponsen vil kunne leses av ved å studere konturene langs enhetssirkelen.

In [5]:
# Filter fra regneeksempel:
b = [1, 1, 1, 1] 
a = [1, -0.6]
plt.close(4); plt.figure(4, figsize=(12,8))
tfPlot(b, a) # Ikke standardfunksjon, definert i kildekoden til forelesningen.

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<img src="Figurer/14_Ztransform/PZ_lecture/Slide2.PNG" style="width: 90%; margin-left: 50px" />

<img src="Figurer/14_Ztransform/PZ_lecture/Slide3.PNG" style="width: 90%; margin-left: 50px" />

<img src="Figurer/14_Ztransform/PZ_lecture/Slide4.PNG" style="width: 90%; margin-left: 50px" />

<img src="Figurer/14_Ztransform/PZ_lecture/Slide5.PNG" style="width: 90%; margin-left: 50px" />

<img src="Figurer/14_Ztransform/PZ_lecture/Slide6.PNG" style="width: 90%; margin-left: 50px" />

<img src="Figurer/14_Ztransform/PZ_lecture/Slide7.PNG" style="width: 90%; margin-left: 50px" />

<img src="Figurer/14_Ztransform/PZ_lecture/Slide8.PNG" style="width: 90%; margin-left: 50px" />

In [6]:
b = [1, 0, 0, 0, 1]
a = [1, 0, 0.49]
visualizeTF(b, a, fig_num=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<img src="Figurer/14_Ztransform/PZ_lecture/Slide9.PNG" style="width: 90%; margin-left: 50px" />

In [13]:
zeroes = np.array([-1,
                   -1j,
                   1j])

poles = np.array([0.7*exp(1j*pi/3),
                  0.7*exp(-1j*pi/3),
                 0])

b, a = zp2tf(zeroes, poles)
visualizeTF(b, a, fig_num=6)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<img src="Figurer/14_Ztransform/PZ_lecture/Slide10.PNG" style="width: 90%; margin-left: 50px" />

<img src="Figurer/14_Ztransform/PZ_lecture/Slide11.PNG" style="width: 90%; margin-left: 50px" />

## Kodeeksempel: Frihåndsdesign av filter

In [8]:
zeroes = np.array([exp(1j*pi/4),
                  exp(-1j*pi/4)])

poles = np.array([0.6*exp(3j*pi/4),
                 0.6*exp(-3j*pi/4)])

b, a = zp2tf(zeroes, poles)
plt.close(6); plt.figure(6)
pzPlot(b, a)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Regneeksempel 4:

* Et filter har transferfunksjonen 
$$H(z) = K \cdot \frac{1 - z^{-1}+ z^{-2}}{1+0.7\cdot z^{-1}+0.49\cdot z^{-2}}$$
der skaleringsfaktoren $K$ ikke er definert.
Identifiser filterformen, og finn en skaleringsfaktor $K$, som fører til at $\left| H\left(e^{j\hat{\omega}}\right)\right| \approx 1$ i passbåndet.

In [14]:
poles = np.roots([1, 0.7, 0.49])
print(np.abs(poles))
print(np.angle(poles)/pi)

[0.7 0.7]
[ 0.66666667 -0.66666667]


In [17]:
b = np.array([1, -1, 1, -1])*0.25
a = np.array([1, 0.7, 0.49])
plt.close(7); plt.figure(7)
Magnitude(b, a)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Kodeeksempel 2: Impulsrespons andreordens filter

* Finn filterkoeffisienter til et filter med to poler i $z = 0.9\cdot e^{\pm j\cdot \frac{\pi}{3}}$, og ett nullpunkt i $z = 0.45$. Vis deretter *impulsresponsen* til filteret i en figur.

In [11]:
zeroes = np.array([0.45])
poles = np.array([0.9*exp(1j*pi/3),
                  0.9*exp(-1j*pi/3)])

b = np.poly(zeroes) 
a = np.poly(poles)

dn = np.zeros(24)
dn[0]=1

hn = sig.lfilter(b, a, dn)

plt.close(7); plt.figure(7)
plt.stem(hn)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<StemContainer object of 3 artists>

# Konvergensområde (ROC)

* Det området hvor z-transformen konvergerer mot en bestemt verdi kalles *Konvergensområde (ROC)*
* Konvergensområdet til en Transferfunksjon $H(z)$ er gitt som området med radius større enn ytterste pol.
    - Eksempel:
    \begin{align}
    y[n] &= x[n] + 0.9 \cdot y[n-1]\\
    h[n] &= 0.9^{n} \\
    Z(h[n]) &= \frac{z}{z-0.9}, \ \ |z| > 0.9
    \end{align}
    

#### <img src="Figurer/14_Ztransform/PZ_lecture/Slide12.PNG" style="width: 90%; margin-left: 50px" />

<img src="Figurer/14_Ztransform/PZ_lecture/Slide13.PNG" style="width: 90%; margin-left: 50px" />

<img src="Figurer/14_Ztransform/PZ_lecture/Slide14.PNG" style="width: 90%; margin-left: 50px" />

<img src="Figurer/14_Ztransform/PZ_lecture/Slide15.PNG" style="width: 90%; margin-left: 50px" />

<img src="Figurer/14_Ztransform/PZ_lecture/Slide16.PNG" style="width: 90%; margin-left: 50px" />

## Kodeeksempel 3: Impulsrespons IIR-filtre
* Lag plot av følgende:
    1. Impulsrespons for et filter med to poler i $z=0.9\cdot e^{\pm j\cdot \frac{\pi}{8}}$
    2. Impulsrespons for et filter med to poler i $z=1.0\cdot e^{\pm j\cdot \frac{\pi}{8}}$
    3. Impulsrespons for et filter med to poler i $z=1.1\cdot e^{\pm j\cdot \frac{\pi}{8}}$

In [12]:
#poles = 0.9*exp(1j*np.array([pi/8, -pi/8])) # Poler innenfor enhetssirkelen
#poles = 1*exp(1j*np.array([pi/8, -pi/8]))     # Poler på enhetssirkelen
poles = 1.1*exp(1j*np.array([pi/8, -pi/8])) # Poler utenfor enhetssirkelen

b, a = zp2tf(poles=poles)

xn = np.zeros(128)
xn[0] = 1
hn = sig.lfilter(b, a, xn)
plt.close(8); plt.figure(8, figsize=(8,3))
hn_plot = plt.stem(hn, markerfmt='.', basefmt='black',use_line_collection=True)
hn_plot.baseline.set_linewidth(0.5)
hn_plot.baseline.set_xdata([0, len(xn)])
plt.xlim([-0.1, len(xn)])
plt.xlabel('Samplenummer $n$')
plt.ylabel(r'$h[n]$')
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<img src="Figurer/14_Ztransform/PZ_lecture/Slide17.PNG" style="width: 90%; margin-left: 50px" />

# Spørsmål?